# Building a Django App and Deploying It on Heroku
> "Building a dummy 'hello world' web app using Django and deploying it on Heroku."

- hide: true
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [mdjango, heroku, github, deployment]
- image: images/django-heroku/heroku-logotype-horizontal-purple.png

In this step-by-step guide, I show how to setup a simple Django web app that says "Hello World!"
and how to deploy it on Heroku through GitHub.


## Requirements

For that, Heroku and GitHub accounts are required. Both are free.

All code should be in a GitHub repository.
For reference, this is my GitHub repository:
https://github.com/stefanbschneider/django-hello-world


## Django "Hello World" App

### Initial Setup

Install Django:

```
pip install django
```

Create a new Django project (inside the GitHub repository):

```
django-admin startproject myproject
```

A project can consist of multiple apps.
Create a new Django `helloworld` app inside the `myproject` project:

```
cd myproject
python manage.py startapp helloworld
```

Run the dev server:

```
python manage.py runserver
```

Go to http://localhost:8000/ in the browser. This should confirm the successful initial setup:

![](/images/django-heroku/django-dev-server.png)


### Create the "Hello World" App


